In [ ]:
import os
HOME = os.getcwd()
print(HOME)

In [ ]:
%cd {HOME}/drive/MyDrive/
!git clone https://github.com/ultralytics/yolov5
%cd {HOME}/drive/MyDrive/yolov5
%pip install -qr requirements.txt

import utils
display = utils.notebook_init()

In [ ]:
%cd {HOME}/drive/MyDrive/yolov5
!python export.py --weights WEIGHT_PATH --include engine --device 0 --img 720 1280  --batch-size 1 --half --verbose --workspace 10 

In [4]:
ENGINE_PATH = "./best.engine"
WEIGHT_PATH = "./best.pt"

In [ ]:
import torch
# load model from torch hub
model = torch.hub.load('.', 'custom', WEIGHT_PATH, source='local')
# model = torch.hub.load('.', 'custom', ENGINE_PATH, source='local')
model.names = ['ball', 'goalkeeper', 'player', 'referee']

In [ ]:
%cd {HOME}
%git clone https://github.com/ifzhang/ByteTrack.git
%cd {HOME}/ByteTrack

# workaround related to https://github.com/roboflow/notebooks/issues/80
%sed -i 's/onnx==1.8.1/onnx==1.9.0/g' requirements.txt

%pip3 install -q -r requirements.txt
%python3 setup.py -q develop
%pip install -q cython_bbox
%pip install -q onemetric
# workaround related to https://github.com/roboflow/notebooks/issues/112
%pip install -q loguru lap
%pip install onemetric --quiet
from IPython import display
display.clear_output()

import sys
sys.path.append(f"{HOME}/ByteTrack")

import yolox
print("yolox.__version__:", yolox.__version__)

In [1]:
import argparse
parser = argparse.ArgumentParser()
parser.add_argument(
    "--source",
    default="./test.mp4",
    type=str,
    help="Path to the input "
)
parser.add_argument(
    "--model", default="/model.pt", type=str, help="Path to the model"
)
parser.add_argument(
    "--fps", default=30, type=int, help="FPS of the video"
)
parser.add_argument(
    "--possession",
    action="store_true",
    help="Enable possession counter"
)
parser.add_argument(
        "--possession_threshold",
        type=int,
        default=10,
        help="Amount of consecutive frames new team has to have the ball in order to change possession"
)
parser.add_argument(
        "--ball_conf",
        type=float,
        default=0.4,
        help="Ball confidence threshold"
)
parser.add_argument(
    "--teams_name",
    nargs='+',
    type=str,
    default=['MUN','SEV'],
    help="Short name of two teams"
)
parser.add_argument(
    "--colors",
    nargs='+',
    type=str,
    default=[],
    help="Colors of two teams and refee, format: [c1,c2,c3,c4,c5] -> [T1,T1,T2,T2,R]"
)
parser.add_argument(
    "--color_t1",
    nargs='+',
    type=str,
    default=['red','blue','sky_blue','blueish_red'], #['sky_blue','orange'], # ['sky_blue','blueish_red','red']
    help="Colors of team 1, format: [c1,c2] -> [T1,T1]"
)
parser.add_argument(
    "--color_t2",
    nargs='+',
    type=str,
    default=['white','yellow'],   #['black','red'], # ['blue','orange']
    help="Colors of team 2, format: [c3,c4] -> [T2,T2]"
)
parser.add_argument(
    "--color_rf",
    type=str,
    default='black', # 'yellow'
    help="Colors of refee, format: [c5] -> [R]"
)
parser.add_argument(
    "--colors_draw",
    nargs='+',
    type=str,
    default=['blue','salmon','yellow'],
    help="Colors to draw annotation base on classification of 2 teams and refee, format: [c1,c2,c3] -> [T1,T2,R]"
)
parser.add_argument(
    "--board_colors",
    nargs='+',
    type=tuple,
    default=['',''],
    help="Colors to draw board possession of 2 teams, format: [c1,c2] -> [T1,T2]"
)
parser.add_argument(
    "--text_colors",
    nargs='+',
    type=tuple,
    default=['',''],
    help="Colors to draw percentage of possession per team , format: [c1,c2] -> [T1,T2]"
)
args = parser.parse_args(args=[])
print(args)

Namespace(source='./test.mp4', model='/model.pt', fps=30, possession=False, possession_threshold=10, ball_conf=0.4, teams_name=['MUN', 'SEV'], colors=[], color_t1=['red', 'blue', 'sky_blue', 'blueish_red'], color_t2=['white', 'yellow'], color_rf='black', colors_draw=['blue', 'salmon', 'yellow'], board_colors=['', ''], text_colors=['', ''])


In [ ]:
import sys
sys.path.insert(0, './football')

In [ ]:

import torch
from analysis.colors import Color
from analysis.filter import filters
from analysis.hsv_classifier import HSVClassifier
from analysis.inertia_classifier import InertiaClassifier
from analysis.possession import (
    get_player_in_possession,
    get_team_in_possession,
    inertia_possession
)

from util.detection import (
    Detection,
    filter_class,
    filter_classification,
    true_ball
)
from util.track import (
    BYTETrackerArgs,
    detections2boxes,
    match_detections_with_tracks
)
from draw.annotate import BaseAnnotator
from draw.marker import MarkerAnntator
from draw.draw_possession import (
    get_possession_background,
    draw_possession_counter
)
from util.video import Video

from yolox.tracker.byte_tracker import BYTETracker

In [ ]:
import matplotlib
# red
HOME_COLOR_HEX = matplotlib.colors.cnames[args.colors_draw[0]]
HOME_COLOR = Color.from_hex_string(HOME_COLOR_HEX)

# green
AWAY_COLOR_HEX = matplotlib.colors.cnames[args.colors_draw[1]]
AWAY_COLOR = Color.from_hex_string(AWAY_COLOR_HEX)

# possession board and text color
HOME_BOARD_HEX = matplotlib.colors.cnames['skyblue']
args.board_colors[0] = Color.from_hex_string(HOME_BOARD_HEX).bgr_tuple
HOME_TEXT_HEX = matplotlib.colors.cnames['white']
args.text_colors[0] = Color.from_hex_string(HOME_TEXT_HEX).bgr_tuple
AWAY_BOARD_HEX = matplotlib.colors.cnames['royalblue']
args.board_colors[1] = Color.from_hex_string(AWAY_BOARD_HEX).bgr_tuple
AWAY_TEXT_HEX = matplotlib.colors.cnames['white']
args.text_colors[1] = Color.from_hex_string(AWAY_TEXT_HEX).bgr_tuple

# yellow
REFEREE_COLOR_HEX = matplotlib.colors.cnames[args.colors_draw[2]]
REFEREE_COLOR = Color.from_hex_string(REFEREE_COLOR_HEX)

In [ ]:
# black
MARKER_CONTOUR_COLOR_HEX = "000000"
MARKER_CONTOUR_COLOR = Color.from_hex_string(MARKER_CONTOUR_COLOR_HEX)

# red
PLAYER_MARKER_FILL_COLOR_HEX = "FF0000"
PLAYER_MARKER_FILL_COLOR = Color.from_hex_string(PLAYER_MARKER_FILL_COLOR_HEX)

# green
BALL_MERKER_FILL_COLOR_HEX = "00FF00"
BALL_MARKER_FILL_COLOR = Color.from_hex_string(BALL_MERKER_FILL_COLOR_HEX)

MARKER_CONTOUR_THICKNESS = 2
MARKER_WIDTH = 15
MARKER_HEIGHT = 15
MARKER_MARGIN = 7

# distance in pixels from the player's bounding box where we consider the ball is in his possession
PLAYER_IN_POSSESSION_PROXIMITY = 45

In [ ]:
SOURCE_VIDEO_PATH = "./samples/mun_sev_test.mp4"
BOARD_IMG_PATH = "./football/draw/possession_board.png"
# SOURCE_VIDEO_PATH = f"{HOME}/drive/MyDrive/mci_rma_test.mp4"

In [ ]:
from typing import Generator

import matplotlib.pyplot as plt
import numpy as np

import cv2

%matplotlib inline

def generate_frames(video_file: str) -> Generator[np.ndarray, None, None]:
    video = cv2.VideoCapture(video_file)

    while video.isOpened():
        success, frame = video.read()

        if not success:
            break

        yield frame

    video.release()


def plot_image(image: np.ndarray, size: int = 12) -> None:
    plt.figure(figsize=(size, size))
    plt.axis('off')
    plt.imshow(image[...,::-1])
    plt.show()

In [ ]:
frame_iterator = iter(generate_frames(video_file=SOURCE_VIDEO_PATH))
frame = next(frame_iterator)
plot_image(frame, 16)

In [ ]:
# initiate video reader and writer
video = Video(input_path=SOURCE_VIDEO_PATH)
args.fps = video.video_capture.get(cv2.CAP_PROP_FPS)


# initiate annotators
THICKNESS = 4
player_home_annotator = BaseAnnotator(
    colors = HOME_COLOR,thickness=THICKNESS)
player_away_annotator = BaseAnnotator(
    colors = AWAY_COLOR,thickness=THICKNESS)
referee_annotator = BaseAnnotator(
    colors = REFEREE_COLOR,thickness=THICKNESS)

ball_marker_annotator = MarkerAnntator(
    color=BALL_MARKER_FILL_COLOR)
player_in_possession_marker_annotator = MarkerAnntator(
    color=PLAYER_MARKER_FILL_COLOR)

# HSV Classifier
filters = filters(args)
hsv_classifier = HSVClassifier(filters=filters)

# Add inertia to classifier
classifier = InertiaClassifier(classifier=hsv_classifier, inertia=20)
possession_background  = get_possession_background(board_img = BOARD_IMG_PATH)

# initiate tracker
byte_tracker = BYTETracker(BYTETrackerArgs())
team_possession = {}


In [ ]:
# loop and run detection over frame
for index, frame in enumerate(video):

    # annotate video frame
    annotated_image = frame.copy()
    # run detector
    results = model(frame[...,::-1], size=1280)
    detections = Detection.from_numpy(
        pred=results.pred[0].cpu().numpy(),
        names=model.names)


    # filter detections by class
    ball_detections = filter_class(detections=detections, class_id=0) 
    ball_detection = true_ball(detections=ball_detections, ball_confidence = args.ball_conf)


    tracked_person_detections = filter_class(detections=detections, class_id=0, reverse=True)
    # tracking
    if len(tracked_person_detections) != 0:
        tracks = byte_tracker.update(
                output_results=detections2boxes(detections=tracked_person_detections),
                img_info=frame.shape,
                img_size=frame.shape
        )
        tracked_detections = match_detections_with_tracks(detections=tracked_person_detections, tracks=tracks)
        tracked_person_detections_pd = classifier.predict_from_detections(detections=tracked_detections.copy(), img=frame)

        tracked_referee_detections = filter_classification(detections = tracked_person_detections_pd, classification="Referee")
        tracked_player_home_detections = filter_classification(detections = tracked_person_detections_pd, classification = args.teams_name[0])
        tracked_player_away_detections = filter_classification(detections = tracked_person_detections_pd, classification = args.teams_name[1])

        player_detections = tracked_player_home_detections + tracked_player_away_detections

        # calculate player in possession and percentage possession per team
        player_in_possession_detection, ball_detection = get_player_in_possession(
                                                         player_detections=player_detections,
                                                         ball_detections=ball_detection,
                                                         proximity=PLAYER_IN_POSSESSION_PROXIMITY)

        team_possession = get_team_in_possession(
                          team_possession = team_possession,
                          player_possession = player_in_possession_detection,
                          args=args)

        player_in_possession_detection = inertia_possession(player_possession = player_in_possession_detection, team_possession = team_possession)

        # annotate frame
        annotated_image = player_home_annotator.annotate(
                          image=annotated_image,
                          detections=tracked_player_home_detections)

        annotated_image = player_away_annotator.annotate(
                          image=annotated_image,
                          detections=tracked_player_away_detections)

        annotated_image = referee_annotator.annotate(
                          image=annotated_image,
                          detections=tracked_referee_detections)

        annotated_image = ball_marker_annotator.annotate(
                          image=annotated_image,
                          detections=ball_detection,
                          width = MARKER_WIDTH,
                          height = MARKER_HEIGHT,
                          margin = MARKER_MARGIN,
                          thickness = MARKER_CONTOUR_THICKNESS,
                          color_contour=MARKER_CONTOUR_COLOR)

        annotated_image = player_in_possession_marker_annotator.annotate(
                          image=annotated_image,
                          detections=[player_in_possession_detection] if player_in_possession_detection else [],
                          width = MARKER_WIDTH,
                          height = MARKER_HEIGHT,
                          margin = MARKER_MARGIN,
                          thickness = MARKER_CONTOUR_THICKNESS,
                          color_contour=MARKER_CONTOUR_COLOR)
        if args.possession:
           annotated_image = draw_possession_counter(
                             team_possession=team_possession,
                             frame=annotated_image.copy(),
                             counter_background=possession_background,
                             args=args)

    # save video frame
    video.write(annotated_image)